In [1]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, BoxSelectTool, DataTable, TableColumn, CDSView, IndexFilter, DateFormatter, DatetimeTickFormatter, NumeralTickFormatter, CustomJS, Panel, Tabs, LinearAxis, Range1d, Paragraph, DatePicker, Div
from bokeh.layouts import gridplot, column, row
from bokeh.io import output_notebook, output_file, curdoc
import bokeh.palettes as palettes
output_notebook()

aeropod_kestrel_file = "PREPROCESSED AEROPOD - Edgley Jan. 13, 2023 - Turbulence Intensity.csv"
ground_kestrel_file = "PREPROCESSED GROUND - Edgley Jan. 13, 2023 - Turbulence Intensity.csv"
test_file = "Edgley Nov. 5, 2022 - Saturday, November 05, 2022 - PREPROCESSED.csv"
test_2 = "Edgley Nov. 5, 2022 - Saturday, November 05, 2022 - Ground - PREPROCESSED.csv"

files = [aeropod_kestrel_file, 
         ground_kestrel_file,
        test_file,
        test_2]

num_of_files = len(files)

#The files and kestrel labels should be ordered correspondingly
kestrel_labels = ["AEROPOD - Jan 13, 2023 ", "GROUND - Jan 13, 2023", "AEROPOD - Nov. 5, 2022", "GROUND - Nov. 5, 2022"]
colors = ["red", "blue", "green", "magenta"] #Red corresponds to the AEROPOD data; Blue corresponds to the GROUND data

opened_file = []
for file in files:
    with open(file) as f:
        opened_file.append(pd.read_csv(f))

#Conversion is necessary, or else Bokeh won't plot the values
time_label = "Time (yyyy-MM-dd hh:mm:ss)"
for df in opened_file:
    df.loc[:,time_label] = pd.to_datetime(df.loc[:,time_label])

###################################################################################################

labels = []
columns = []

#The columns are listed in the order of the desired plot layout

alt_label = "AOG (Meters)"
#alt_label = "AOG (Feet)"
labels.append(alt_label)
columns.append("alt")

ws_label = "Wind Speed (m/s)"
#ws_label = "Wind Speed (mph)"
#ws_label = "Wind Speed (ft/s)",
#ws_label = "Wind Speed (kt)",
#ws_label = "Wind Speed (Bft)",
labels.append(ws_label)
columns.append("ws")

temp_label = "Temp (Celsius)"
#temp_label = "Temp (Fahrenheit)"
labels.append(temp_label)
columns.append("temp")

wbtemp_label = "Wet Bulb Temp. (Celsius)"
labels.append(wbtemp_label)
columns.append("wetbulbtemp")

rh_label = "Rel. Hum. (%)"
labels.append(rh_label)
columns.append("rh")

baro_label = "Baro. (mb)"
labels.append(baro_label)
columns.append("baro")

magdir_label = "Mag. Dir. (Degrees)"
labels.append(magdir_label)
columns.append("magdir")

elapsed_time_label = "Elapsed Time (seconds)"

sources = []

for i in range(num_of_files):
    current_file = files[i]
    print(f"Loading from {current_file}")
    col_count = 0
    df = opened_file[i]
    num_entries = len(df)    
    dic = {
        "index": df.index,
        "time": df[time_label],
        "elapsed_time": df[elapsed_time_label],
        "elapsed_time_copy": df[elapsed_time_label].copy()
    }
    
    for col, label in zip(columns, labels):
        if label in df.columns:
            dic[col] = df[label]
            col_count += 1
            print(f"{label} was loaded")
        else:
            print(f"{label} does not exist in this file")            
    
    source = ColumnDataSource(data=dic)
    print(f"{col_count} columns were loaded from {current_file}\n")
    
    sources.append(source)        

Loading BokehJS ...

Loading from PREPROCESSED AEROPOD - Edgley Jan. 13, 2023 - Turbulence Intensity.csv
AOG (Meters) was loaded
Wind Speed (m/s) was loaded
Temp (Celsius) was loaded
Wet Bulb Temp. (Celsius) was loaded
Rel. Hum. (%) was loaded
Baro. (mb) was loaded
Mag. Dir. (Degrees) was loaded
7 columns were loaded from PREPROCESSED AEROPOD - Edgley Jan. 13, 2023 - Turbulence Intensity.csv

Loading from PREPROCESSED GROUND - Edgley Jan. 13, 2023 - Turbulence Intensity.csv
AOG (Meters) was loaded
Wind Speed (m/s) was loaded
Temp (Celsius) was loaded
Wet Bulb Temp. (Celsius) was loaded
Rel. Hum. (%) was loaded
Baro. (mb) was loaded
Mag. Dir. (Degrees) was loaded
7 columns were loaded from PREPROCESSED GROUND - Edgley Jan. 13, 2023 - Turbulence Intensity.csv

Loading from Edgley Nov. 5, 2022 - Saturday, November 05, 2022 - PREPROCESSED.csv
AOG (Meters) was loaded
Wind Speed (m/s) was loaded
Temp (Celsius) was loaded
Wet Bulb Temp. (Celsius) was loaded
Rel. Hum. (%) was loaded
Baro. (mb) was loaded
Mag. Dir.

# Generate plots

## Time series plots

In [2]:
time_shift = [0, 0, 0, 0]

for i in range(len(time_shift)):
    sources[i].data["elapsed_time_copy"] += time_shift[i]

#Might as well generate random colors unless you want to specify a color for EVERY single field for EVERY single datafile
#import random 

#output_file("Plotting Multiple Kestrels - Turbulence Intensity - Edgley Jan. 13, 2023.html")

#Initialize the colors list with num_of_files different colors
#colors = [i for i in range(num_of_files)]

#Plot formatting

#x = "time"
x = "elapsed_time_copy"

hover = HoverTool( #API Reference: https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#hovertool
    tooltips=[
        ("Index", "$index"),
        ("Time", "@time{%F %I:%M:%S %p}"),
        ("Value", "$y"),
        ("X", "$x"),
    ],

    formatters={
        "@time" : "datetime",
    },
    #mode = "vline"
)

time_series_options = dict(tools=[hover, "pan, wheel_zoom, box_select, tap, reset"], plot_width=800, plot_height=400)

timefigures = []
plot_layout = []

sz = 2

#Plot generation
for label, col in zip(labels, columns):
    if x == "time":    
        timefigures.append(figure(title=f"{label} vs. Time", x_axis_label = "Time", 
                                  y_axis_label=label, x_axis_type = "datetime", 
                                  **time_series_options))    
        
        timefigures[-1].xaxis.formatter=DatetimeTickFormatter(
            hours="%I:%M:%S %p",
            minutes="%I:%M:%S %p")        
        
    elif x == "elapsed_time_copy":
        timefigures.append(figure(title=f"{label} vs. Elapsed Time", 
                                  x_axis_label = elapsed_time_label, 
                                  y_axis_label=label, 
                                  **time_series_options))      

    ####################################################################################################################
    if label == "Mag. Dir. (Degrees)":
        timefigures[-1].y_range.start = -40
        timefigures[-1].y_range.end = 400
        
        for i in range(num_of_files):    
            timefigures[-1].scatter(x, col, color=colors[i], legend_label = f"{kestrel_labels[i]}", size=sz, source=sources[i])
    
    #This section is where to add the plot renderers for n number of dataframes
    else:
        for i in range(num_of_files):
            #colors[i] = ("#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)]))
            timefigures[-1].line(x, col, color=colors[i], legend_label = f"{kestrel_labels[i]}", source=sources[i])
            timefigures[-1].scatter(x, col, color=colors[i], legend_label = f"{kestrel_labels[i]}", size=sz, source=sources[i])
    
    
    ####################################################################################################################
    
    timefigures[-1].title.text_color = "black"
    timefigures[-1].background_fill_color = "beige"
    
    timefigures[-1].x_range = timefigures[0].x_range
    
    timefigures[-1].legend.click_policy = "hide"

#tab1 = Panel(child=column(plot_layout), title="Time Series Plots")

#layout1 = Tabs(tabs=[tab1])

show(column(timefigures))

## Alt profiles

In [3]:

#Might as well generate random colors unless you want to specify a color for EVERY single field for EVERY single datafile
#import random 

#output_file("Plotting Multiple Kestrels - Turbulence Intensity - Edgley Jan. 13, 2023.html")

#Initialize the colors list with num_of_files different colors
#colors = [i for i in range(num_of_files)]

hover = HoverTool( #API Reference: https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#hovertool
    tooltips=[
        ("Index", "$index"),
        ("Time", "@time{%F %I:%M:%S %p}"),
        ("Value", "$y"),
        ("X", "$x"),
    ],

    formatters={
        "@time" : "datetime",
    },
    #mode = "vline"
)

alt_profiles_options = dict(tools=[hover, "pan, wheel_zoom, box_select, tap, reset"], plot_width=800, plot_height=400)

altfigures = []
plot_layout = []


#Creating copies to allow this cell to be rerun multiple times
labels_altprofiles = labels.copy()
labels_altprofiles.remove(alt_label) 

columns_altprofiles = columns.copy()
columns_altprofiles.remove("alt")

sz = 2



for label, col in zip(labels_altprofiles, columns_altprofiles):

    altfigures.append(figure(title=f"{label} vs. {alt_label}", x_axis_label = alt_label, 
                              y_axis_label=label,
                              **alt_profiles_options))
    
    ####################################################################################################################
    #This section is where to add the plot renderers for n number of dataframes
            
    for i in range(num_of_files):    
        altfigures[-1].circle("alt", col, color=colors[i], legend_label = f"{kestrel_labels[i]} - {label}", size=sz, source=sources[i])

    
    ####################################################################################################################
    
    altfigures[-1].title.text_color = "black"
    altfigures[-1].background_fill_color = "beige"
    
    altfigures[-1].x_range = altfigures[0].x_range
    
    #altfigures[-1].legend.click_policy = "hide"

#tab1 = Panel(child=column(plot_layout), title="Time Series Plots")

#layout1 = Tabs(tabs=[tab1])

show(column(altfigures))

## Plotting Wet Bulb Temperature on the same plot as temperature

In [4]:
import random #To generate a random color
#output_file("Plotting Multiple Kestrels - Turbulence Intensity - Edgley Jan. 13, 2023.html")

#Initialize the colors list with num_of_files different colors
wetbulbcolors = [i for i in range(num_of_files)]

hover = HoverTool( #API Reference: https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#hovertool
    tooltips=[
        ("Index", "$index"),
        ("Time", "@time{%F %I:%M:%S %p}"),
        ("Value", "$y"),
        ("X", "$x"),
    ],

    formatters={
        "@time" : "datetime",
    },
    #mode = "vline"
)

time_series_options = dict(tools=[hover, "pan, wheel_zoom, box_select, hover, tap, reset"], plot_width=800, plot_height=400)

timefigures = []
plot_layout = []

sz = 2

for label, col in zip(labels, columns):

    if label != wbtemp_label:
        
        if x == "time":
            timefigures.append(figure(title=f"{label} vs. Time", 
                                      x_axis_label = "Time",
                                      x_axis_type = "datetime", 
                                      y_axis_label=label,
                                      **time_series_options))
            
            timefigures[-1].xaxis.formatter=DatetimeTickFormatter(
              hours="%I:%M:%S %p",
              minutes="%I:%M:%S %p")    
            
        elif x == "elapsed_time_copy":
            timefigures.append(figure(title=f"{label} vs. Elapsed Time", 
                                      x_axis_label = elapsed_time_label, 
                                      y_axis_label=label, 
                                      **time_series_options))

        ####################################################################################################################
        #This section is where to add the plot renderers for n number of dataframes
        if label == "Mag. Dir. (Degrees)":
            timefigures[-1].y_range.start = -40
            timefigures[-1].y_range.end = 400
            

            for i in range(num_of_files):
                #colors[i] = ("#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)]))   
                timefigures[-1].circle(x, col, color=colors[i], legend_label = f"{kestrel_labels[i]} - {label}", size=sz, source=sources[i])

        else:
            for i in range(num_of_files):
                timefigures[-1].line(x, col, color=colors[i], legend_label = f"{kestrel_labels[i]} - {label}", source=sources[i])
                timefigures[-1].circle(x, col, color=colors[i], legend_label = f"{kestrel_labels[i]} - {label}", size=sz, source=sources[i])
                
                if label == temp_label:
                    wetbulbcolors[i] = ("#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)]))                
                    timefigures[-1].line(x, "wetbulbtemp", color=wetbulbcolors[i], legend_label = f"{kestrel_labels[i]} - {wbtemp_label}", source=sources[i])
                    timefigures[-1].circle(x, "wetbulbtemp", color=wetbulbcolors[i], legend_label = f"{kestrel_labels[i]} - {wbtemp_label}", source=sources[i])


        ####################################################################################################################

        timefigures[-1].title.text_color = "black"
        timefigures[-1].background_fill_color = "beige"

        timefigures[-1].x_range = timefigures[0].x_range

        timefigures[-1].legend.click_policy = "hide"

#tab1 = Panel(child=column(plot_layout), title="Time Series Plots")

#layout1 = Tabs(tabs=[tab1])

show(column(timefigures))